In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("spotify_millsongdata.csv")

In [4]:
df.head(10)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
5,ABBA,Burning My Bridges,/a/abba/burning+my+bridges_20003011.html,"Well, you hoot and you holler and you make me ..."
6,ABBA,Cassandra,/a/abba/cassandra_20002811.html,Down in the street they're all singing and sho...
7,ABBA,Chiquitita,/a/abba/chiquitita_20002978.html,"Chiquitita, tell me what's wrong \r\nYou're e..."
8,ABBA,Crazy World,/a/abba/crazy+world_20003013.html,I was out with the morning sun \r\nCouldn't s...
9,ABBA,Crying Over You,/a/abba/crying+over+you_20177611.html,I'm waitin' for you baby \r\nI'm sitting all ...


In [37]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [38]:
df.head(10)

,artist,song,text
0,Dean Martin,Deep Purple,when the deep purple falls \r over sleepy gar...
1,Perry Como,Ain't Misbehavin',no one to talk with all by myself \r no one t...
2,Yellowcard,Lights And Sounds,"hello you, how was the rest, \r you made it t..."
3,Eddie Cochran,Mojo Workout,"i got my mojo workin', well my mojo workin' \..."
4,Grand Funk Railroad,Trying To Get Away,i been out on the road with my thumb in the ai...
5,Christy Moore,Joxer goes to Stuttgart,it was in the year of '88 in the lovely month ...
6,Linda Ronstadt,I Just Don't Know What To Do With Myself,i just don't know what to do with myself \r d...
7,Joni Mitchell,Stormy Weather,don't know why \r there's no sun up in the sk...
8,Britney Spears,Piece Of Me,i'm miss american dream since i was seventeen ...
9,Bonnie Raitt,Two Lights In The Nighttime,no river can hold you darlin' \r no covers ca...


In [40]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [43]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Rithi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [44]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)


In [45]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [48]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)


In [49]:
similarity[0]

array([1.        , 0.02289596, 0.00634911, ..., 0.00394051, 0.0464652 ,
       0.0952619 ])

In [55]:
df[df['song'] == 'Beautiful']

,artist,song,text
223,Prince,Beautiful,the most beauti girl in the world beauti beaut...
4011,Amy Grant,Beautiful,i 'm lookin ' for a way to feel you hold me fe...


In [56]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs


In [58]:
recommendation('Beautiful')


['Beautiful World',
 'Beautiful Disaster',
 "Oh What A Beautiful Mornin'",
 'Beautiful Noise',
 'Beautiful In This Mirror',
 'The Beauty Of That',
 'Beautiful',
 'Oh, What A Beautiful Name',
 'Here She Comes',
 'Bad And Beautiful',
 'The Beauty In Ugly',
 'Beautiful Savior',
 'Be There',
 "Isn't It A Pity",
 "It's Gonna Be A Good Day",
 'Girls',
 'Try Try Try',
 'Girl Happy',
 'Marry You',
 'America The Beautiful']

In [59]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))